In [1]:
%pip install gensim
%pip install torch
%pip install Pillow
%pip install requests
%pip install open_clip_torch
%pip install torchvision


  Using cached gensim-4.3.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (8.3 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.17.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.4 kB)
Using cached gensim-4.3.3-cp39-cp39-macosx_11_0_arm64.whl (24.0 MB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl (30.3 MB)
Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
Using cached wrapt-1.17.2-cp39-cp39-macosx_11_0_arm64.whl (38 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached torch-2.6.0-cp39-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jinja2-3.1.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import gensim.downloader as api # pip install gensim
from txt2png import txt_to_png
from open_clip_test_mps import clip_loss, device, preprocess

/Users/dalfr76/Documents/CS_673/ASCII_animation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/dalfr76/Documents/CS_673/ASCII_animation/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device for image: mps
Using device for text: cpu


In [3]:

word_model_name="glove-twitter-25"
word_model = api.load(word_model_name)

embedding_dim = 25

output_size = 15 * 6


ascii_chars = " .,:;+*#@$%&0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ-_=|/\\()[]{}"
num_chars = len(ascii_chars)
char_to_index = {char: i for i, char in enumerate(ascii_chars)}
index_to_char = {i: char for i, char in enumerate(ascii_chars)}

class ASCIIArtGenerator(nn.Module):
    def __init__(self, input_size, output_size, num_chars):
        super(ASCIIArtGenerator, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_size * num_chars)
        self.num_chars = num_chars
        self.output_size = output_size

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x.view(-1, self.output_size, self.num_chars) 

def generate_ascii_art(model, embedding):
    model.eval()
    with torch.no_grad():
        output = model(embedding.unsqueeze(0))
        _, indices = torch.max(output, dim=2)
        ascii_grid = ""
        for i in range(6):
            for j in range(15):
                ascii_grid += index_to_char[indices[0][i * 15 + j].item()]
            ascii_grid += "\n"
    return ascii_grid


model = ASCIIArtGenerator(embedding_dim, output_size, num_chars)

In [5]:
embedding = torch.tensor(word_model["cat"])
for i in range(10):
    optimizer = optim.Adam(model.parameters(), lr = .01)
    ascii_art = generate_ascii_art(model, embedding)
    ascii_png = txt_to_png(ascii_art, output_png="temp.png", save=True)
    #image = Image.open("./Data/Image/castle_710.png")
    image_tensor = preprocess(ascii_png).unsqueeze(0).to(device, dtype=torch.float32)
    optimizer.zero_grad()
    #generated_images = generator(noise)          # Your generator produces images.
    candidate_labels = ["a bird", "a dog", "a cat", "a castle"]
    true_label = 2
    loss = clip_loss(image_tensor, candidate_labels, true_label)
    loss.backward()
    optimizer.step()
    print(ascii_art)
    print()

b(hS+x4GOz[4gMA
cV)yCWTw4In;Vb&
%gqjGD& |5E;0_B
Fd3zAcZ-g%53:iF
oDF3-=_-[)ywcxP
w_4UuS#WG;m@%R_


b(hS+x4GOz[4gMA
cV)yCWTw4In;Vb&
%gqjGD& |5E;0_B
Fd3zAcZ-g%53:iF
oDF3-=_-[)ywcxP
w_4UuS#WG;m@%R_


b(hS+x4GOz[4gMA
cV)yCWTw4In;Vb&
%gqjGD& |5E;0_B
Fd3zAcZ-g%53:iF
oDF3-=_-[)ywcxP
w_4UuS#WG;m@%R_




KeyboardInterrupt: 